In [179]:
import os

In [180]:
%pwd

'C:\\datascience\\Projects\\text_summarizer'

In [181]:
import os
os.chdir(r"C:\datascience\Projects\text_summarizer")


In [182]:
%pwd

'C:\\datascience\\Projects\\text_summarizer'

In [183]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [184]:
from textSummarizer.constants import *
from textSummarizer.utils.common import create_directory,read_yaml_config,get_size
from textSummarizer.logger_config.config import logger
import urllib.request as request
import zipfile


In [185]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml_config(config_filepath)
        self.params = read_yaml_config(params_filepath)

        create_directory([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directory([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [ ]:
import py7zr
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file: {filename} with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with py7zr.SevenZipFile(self.config.local_data_file, mode='r') as z:
            z.extractall(path=unzip_path)
            logger.info(f"Extracted 7z file to: {unzip_path}")


            
        

In [187]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-04-05 01:46:49,539: INFO: Successfully loaded configuration from: config\config.yaml]
[2025-04-05 01:46:49,541: INFO: Successfully loaded configuration from: params.yaml]
[2025-04-05 01:46:49,543: INFO: Created directory at: artifacts]
[2025-04-05 01:46:49,544: INFO: Created directory at: artifacts/data_ingestion]
[2025-04-05 01:46:51,224: INFO: Downloaded file: artifacts/data_ingestion/data.zip with following info: 
Connection: close
Content-Length: 2944100
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/octet-stream
ETag: "4450eb63089f56212a41409ae9bcb15b3fb051149cf8d4b895f422d5c86b831c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4B48:0840:9B658:164D23:67F03C94
Accept-Ranges: bytes
Date: Fri, 04 Apr 2025 20:16:29 GMT
Via: 1.1 varnish
X-Served-By: cache-del21727-DEL
X-Cache: HIT
X-Cache-Hi

In [188]:
from textSummarizer.utils.common import read_yaml_config
from pathlib import Path

cfg = read_yaml_config(Path("config/config.yaml"))
print(cfg)


[2025-04-05 01:46:51,429: INFO: Successfully loaded configuration from: config\config.yaml]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/arpit000000/text_summarizer/raw/refs/heads/main/research/corpus.7z', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
